In [1]:
# Download dataset
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip 

--2022-06-27 11:23:15--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.15.112, 142.251.16.128, 142.251.45.112, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.15.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘cats_and_dogs_filtered.zip’

cats_and_dogs_filte 100%[===================>]  65.43M   339MB/s    in 0.2s    

2022-06-27 11:23:15 (339 MB/s) - ‘cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [2]:
!unzip \cats_and_dogs_filtered.zip &> /dev/null 

### Masuk ke lokasi tempat menyimpan dataset

In [3]:
%cd /content/drive/My Drive/RESEARCH CENTER/Kaggle

[Errno 2] No such file or directory: '/content/drive/My Drive/RESEARCH CENTER/Kaggle'
/content


### Definisi data generator

In [4]:
from keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   rescale=1./255,
                                   shear_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

training_set = train_datagen.flow_from_directory("cats_and_dogs_filtered/train/",
                                                 target_size=(128,128),
                                                 color_mode="rgb",
                                                 shuffle=False,
                                                 batch_size = 8,
                                                 class_mode='categorical')

Found 2000 images belonging to 2 classes.


In [5]:
validation_datagen = ImageDataGenerator(width_shift_range=0.2,
                                        height_shift_range=0.2,
                                        rescale=1./255,
                                        shear_range=0.2,
                                        horizontal_flip=True,
                                        fill_mode='nearest')

validation_set = validation_datagen.flow_from_directory("cats_and_dogs_filtered/validation/",
                                                        target_size=(128,128),
                                                        color_mode="rgb",
                                                        shuffle=False,
                                                        batch_size = 8,
                                                        class_mode='categorical')

Found 1000 images belonging to 2 classes.


### Load TensorBoard notebook extension dan Hapus semua **logs** sebelumnya

In [6]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

In [7]:
%load_ext tensorboard

In [8]:
!rm -rf ./logs/ 

### Definisikan hparam yang ingin digunakan. 

Semakin banyak hparam yang didefinisikan maka semakin banyak pula kombinasi hyperparameter digunakan untuk training model. Semakin banyak kombinasi maka semakin memakan banyak waktu (FYI : lama training model yang digunakan dalam contoh ini +/- 3-4 jam menggunakan GPU).

Menyimpan semua summary hparam kedalam `logs/hparam_tuning` untuk digunakan oleh tensorboard dan sebagai penyimpanan sementara training information.

In [9]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([64, 128]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.2, 0.5))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd', 'adadelta']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

### Definisikan `x_val dan y_val` dari validation data untuk evaluasi model.

In [11]:
import numpy as np
xvl, yvl = zip(*(validation_set[i] for i in range(len(validation_set))))
x_val, y_val = np.vstack(xvl), np.vstack(yvl)

### Definisikan Model menggunakan hparam.

Disini anda mendefinisikan model, compile model, training model secara bersamaan didalam fungsi `train_test_model`. Fungsi ini akan mengembalikan nilai akurasi rata-rata dari masing-masing model.

In [12]:
def train_test_model(hparams):
  model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(64, (5, 5), 
                                                             activation=tf.nn.relu, 
                                                             input_shape=(128,128,3)),
                                      tf.keras.layers.BatchNormalization(),
                                      tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
                                      tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
                                      tf.keras.layers.Conv2D(64, (5, 5), 
                                                             activation=tf.nn.relu),
                                      tf.keras.layers.BatchNormalization(),
                                      tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
                                      tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
                                      tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
                                      tf.keras.layers.Conv2D(32, (5, 5), 
                                                             activation=tf.nn.relu),
                                      tf.keras.layers.BatchNormalization(),
                                      tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
                                      tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
                                      tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
                                      tf.keras.layers.Flatten(),
                                      tf.keras.layers.Dense(hparams[HP_NUM_UNITS], 
                                                            activation=tf.nn.relu),
                                      tf.keras.layers.Dense(64, activation=tf.nn.relu),
                                      tf.keras.layers.Dense(32, activation=tf.nn.relu),
                                      tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
                                      tf.keras.layers.Dense(2, activation=tf.nn.softmax),
                                      ])

  model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='categorical_crossentropy',
      metrics=['accuracy'],
  )

  model.fit(training_set, 
            validation_data=validation_set, 
            epochs=25,
            )
  _, accuracy = model.evaluate(x_val, y_val)
  return accuracy

### Definisikan fungsi `run` 

Hal ini bertujuan untuk melakukan log record hparams yang berisi summary dari hyperparameters dan final accuracy-nya.

In [13]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=50)

### Grid Search Training Model


In [14]:
['adam', 'sgd', 'adadelta', 'rmsprop', 'adamx', 'nadam']

['adam', 'sgd', 'adadelta', 'rmsprop', 'adamx', 'nadam']

In [ ]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_DROPOUT: dropout_rate,
          HP_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1


--- Starting trial: run-0
{'num_units': 64, 'dropout': 0.2, 'optimizer': 'adadelta'}
Epoch 1/25
250/250 [==============================] - 31s 77ms/step - loss: 1.1317 - accuracy: 0.5045 - val_loss: 0.7221 - val_accuracy: 0.4940
Epoch 2/25
250/250 [==============================] - 19s 76ms/step - loss: 1.1074 - accuracy: 0.4930 - val_loss: 0.7862 - val_accuracy: 0.5070
Epoch 3/25
250/250 [==============================] - 19s 75ms/step - loss: 1.0562 - accuracy: 0.5090 - val_loss: 0.8111 - val_accuracy: 0.4990
Epoch 4/25
250/250 [==============================] - 19s 76ms/step - loss: 1.0783 - accuracy: 0.4935 - val_loss: 0.7842 - val_accuracy: 0.4970
Epoch 5/25
250/250 [==============================] - 20s 82ms/step - loss: 1.0585 - accuracy: 0.5030 - val_loss: 0.7815 - val_accuracy: 0.5150
Epoch 6/25
250/250 [==============================] - 20s 78ms/step - loss: 1.0255 - accuracy: 0.4950 - val_loss: 0.7896 - val_accuracy: 0.4930
Epoch 7/25
250/250 [==============================]

### Visualisasi hasil evaluasi semua model dengan hyperparameternya menggunakan Tensorboard

In [ ]:
%%bash
wget -q 'https://storage.googleapis.com/download.tensorflow.org/tensorboard/hparams_demo_logs.zip'
unzip -q hparams_demo_logs.zip -d logs/hparam_demo

In [ ]:
%tensorboard --logdir logs/hparam_tuning

In [ ]:
!kill 5496